In [2]:
import os, glob, math, random, json, warnings, itertools, time, gc
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, classification_report
from sklearn.utils.class_weight import compute_class_weight

import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR

warnings.filterwarnings('ignore')
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)

# ------------------------------------------------------------------
RAW_DIR   = Path('data')
TRAIN_DIR = RAW_DIR / 'train'
TRAIN_CSV = sorted(TRAIN_DIR.glob('sbj_*.csv'))
TEST_CSV  = RAW_DIR / 'test.csv'
LOCATION_IDS = ["right_arm", "left_arm", "right_leg", "left_leg"]
WORK_DIR = Path('work1')
WIN_SIZE  = 50
STRIDE    = 25
BATCH_SZ  = 256
EPOCHS    = 20
EMB_DIM   = 64
LR        = 1e-3
NUM_WORKERS = 4
DEVICE    = 'cuda' if torch.cuda.is_available() else 'cpu'
label_map = {
    'null': 0,'jogging': 1,'jogging (rotating arms)': 2,'jogging (skipping)': 3,'jogging (sidesteps)': 4,'jogging (butt-kicks)': 5,
    'stretching (triceps)': 6,'stretching (lunging)': 7,'stretching (shoulders)': 8,'stretching (hamstrings)': 9,'stretching (lumbar rotation)': 10,
    'push-ups': 11,'push-ups (complex)': 12,'sit-ups': 13,'sit-ups (complex)': 14,'burpees': 15,'lunges': 16,'lunges (complex)': 17,'bench-dips': 18
}
loc = "right_arm"

### 1. Generate per‑location CSVs (adds `subject_id`)

In [11]:
def make_location_csvs():
    WORK_DIR.mkdir(parents=True, exist_ok=True)
    for loc in LOCATION_IDS:
        cols = [f"{loc}_acc_{ax}" for ax in ['x','y','z']] + ['label']
        frames = []
        for csv_file in TRAIN_CSV:
            df = pd.read_csv(csv_file, usecols=lambda c: c in cols)
            subj_id = int(Path(csv_file).stem.split('_')[-1])
            df['subject_id'] = subj_id
            df = df.dropna(subset=[f"{loc}_acc_x", f"{loc}_acc_y", f"{loc}_acc_z", 'label'])
            frames.append(df)
        combined = pd.concat(frames, ignore_index=True)
        out_path = WORK_DIR / f"train_{loc}.csv"
        combined.to_csv(out_path, index=False)
        print(f"Written {out_path}: {len(combined)} rows")


make_location_csvs()
print("Generated 4 train CSVs in WORK_DIR.")

Written work1/train_right_arm.csv: 2089038 rows
Written work1/train_left_arm.csv: 2054764 rows
Written work1/train_right_leg.csv: 2089038 rows
Written work1/train_left_leg.csv: 2089038 rows
Generated 4 train CSVs in WORK_DIR.


### 2. Dataset & augmentations

In [3]:

def jitter(x, sigma=0.015):
    return x + torch.randn_like(x) * sigma

def scaling(x, sigma=0.1):
    factor = torch.normal(1.0, sigma, (x.size(0), 1), device=x.device)
    return x * factor

def rotation(x):
    B, C, T = x.shape
    angles = torch.randn(B, 3, device=x.device) * 0.2
    Rx = torch.tensor([[1,0,0],[0, torch.cos(angles[:,0]), -torch.sin(angles[:,0])],
                       [0, torch.sin(angles[:,0]),  torch.cos(angles[:,0])]])
    return x

class WearWindowDataset(Dataset):
    def __init__(self, df, scaler=None, train=True, augment=True):
        self.train = train
        self.augment = augment and train

        loc = df.columns[0].split('_acc_')[0]

        feats = df[[f"{loc}_acc_x", f"{loc}_acc_y", f"{loc}_acc_z"]].values.astype('float32')

        labels = df['label'].values.astype('int64') if 'label' in df.columns else None
        X, y = [], []
        for i in range(0, len(feats) - WIN_SIZE + 1, STRIDE):
            window = feats[i : i + WIN_SIZE]                # shape (50, 3)
            if np.isnan(window).any():
                continue
            X.append(window)                                # still (50, 3)
            if labels is not None:
                y.append(np.bincount(labels[i : i + WIN_SIZE]).argmax())

        self.X = np.stack(X)                                # (N_windows, 50, 3)
        self.y = np.array(y) if labels is not None else None

        if scaler is not None:
            flat = self.X.reshape(-1, 3)
            flat = scaler.transform(flat)
            self.X = flat.reshape(-1, WIN_SIZE, 3)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        # from (50,3) to (3,50) for Conv1d
        x = torch.tensor(self.X[idx]).permute(1, 0)   # now (3,50)
        if self.augment:
            x = jitter(x)
            x = scaling(x)
        if self.y is None:
            return x
        return x, torch.tensor(self.y[idx])



### 3. Shallow DeepConvLSTM model

In [4]:

class ShallowDeepConvLSTM(nn.Module):
    def __init__(self, n_classes, emb_dim=EMB_DIM):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(3, 64, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.Conv1d(64, 64, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.BatchNorm1d(64)
        )
        self.lstm = nn.LSTM(input_size=64, hidden_size=128, num_layers=1, batch_first=True)
        self.embed = nn.Sequential(nn.Dropout(0.3), nn.Linear(128, emb_dim), nn.ReLU(inplace=True))
        self.head  = nn.Linear(emb_dim, n_classes)
    def forward(self, x):          # x (B, C=3, T=50)
        x = self.conv(x)           # (B, 64, T)
        x = x.permute(0,2,1)       # (B, T, 64)
        _, (h, _) = self.lstm(x)   # h (1, B, 128)
        emb = self.embed(h[-1])    # (B, emb_dim)
        return self.head(emb)


### 4. Focal loss & training utilities

In [5]:

class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.ce = nn.CrossEntropyLoss(reduction='none', weight=alpha)
    def forward(self, logits, target):
        ce_loss = self.ce(logits, target)
        pt = torch.exp(-ce_loss)
        focal = ((1-pt) ** self.gamma) * ce_loss
        return focal.mean()

def run_epoch(model, loader, optim=None, scheduler=None, criterion=None):
    train = optim is not None
    model.train(train)
    losses = []
    for x, y in loader:
        x, y = x.to(DEVICE), y.to(DEVICE)

        with torch.set_grad_enabled(train):
            logits = model(x)
            loss   = criterion(logits, y)
            if train:
                optim.zero_grad()
                loss.backward()
                optim.step()
        losses.append(loss.item())

    if scheduler and train:
        scheduler.step()
    return float(np.mean(losses))


### 5. Training ‑‑ Leave‑One‑Subject‑Out CV

In [6]:

def train_location(location_csv, location):
    df = pd.read_csv(location_csv)
    df['label'] = df['label'].map(label_map).astype(int)
    subjects = df['subject_id'].unique()
    fold_f1 = []

    for val_subj in subjects:
        train_df = df[df.subject_id != val_subj].reset_index(drop=True)
        val_df   = df[df.subject_id == val_subj].reset_index(drop=True)
        scaler   = StandardScaler().fit(train_df.filter(like=f"{location}_acc_"))
        train_ds = WearWindowDataset(train_df, scaler, train=True, augment=True)
        val_ds   = WearWindowDataset(val_df,   scaler, train=False, augment=False)
        counts = np.bincount(train_ds.y, minlength=len(label_map))
        sampler =  torch.utils.data.WeightedRandomSampler(
            weights=1.0 / np.maximum(counts, 1),
            num_samples=len(train_ds.y),
            replacement=True
        )
        train_dl = DataLoader(train_ds, batch_size=BATCH_SZ, sampler=sampler, shuffle=False,num_workers=NUM_WORKERS, pin_memory=True)
        val_dl   = DataLoader(val_ds,   batch_size=BATCH_SZ, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

        n_classes = len(label_map)
        model     = ShallowDeepConvLSTM(n_classes).to(DEVICE)
        optim     = AdamW(model.parameters(), lr=LR, weight_decay=1e-4)
        sched     = CosineAnnealingLR(optim, T_max=EPOCHS)
        loss_fn   = FocalLoss(alpha=None)

        for epoch in range(1, EPOCHS+1):
            loss = run_epoch(model, train_dl, optim, sched, loss_fn)
        y_true, y_pred = [], []
        with torch.no_grad():
            for x, y in val_dl:
                x = x.to(DEVICE)
                logits = model(x)
                y_true.append(y.numpy())
                y_pred.append(logits.argmax(1).cpu().numpy())
        y_true = np.concatenate(y_true)
        y_pred = np.concatenate(y_pred)
        f1 = f1_score(y_true, y_pred, average="macro")
        fold_f1.append(f1)

        ckpt = {
            "state_dict": model.state_dict(),
            "scaler_mean": scaler.mean_,
            "scaler_scale": scaler.scale_
        }
        ckpt_path = WORK_DIR / f"model_{location}_fold{val_subj}.pt"
        torch.save(ckpt, ckpt_path)

        print(f"→ {location} | held-out subj {val_subj} | F1={f1:.4f}")

    mean_f1 = float(np.mean(fold_f1))
    print(f"=== {location} LOSO mean F1: {mean_f1:.4f} ===")
    return mean_f1


In [7]:

location_scores = {}
#for loc in LOCATION_IDS:
score = train_location(WORK_DIR/f'train_{loc}.csv','right_arm')
location_scores[loc] = score
print('LOSO scores per location:', location_scores)


→ right_arm | held-out subj 0 | F1=0.0103
→ right_arm | held-out subj 2 | F1=0.0072
→ right_arm | held-out subj 1 | F1=0.0219
→ right_arm | held-out subj 10 | F1=0.0105
→ right_arm | held-out subj 11 | F1=0.0231
→ right_arm | held-out subj 12 | F1=0.0219
→ right_arm | held-out subj 13 | F1=0.0164
→ right_arm | held-out subj 14 | F1=0.0099
→ right_arm | held-out subj 15 | F1=0.0091
→ right_arm | held-out subj 16 | F1=0.0138
→ right_arm | held-out subj 17 | F1=0.0243
→ right_arm | held-out subj 18 | F1=0.0120
→ right_arm | held-out subj 19 | F1=0.0244
→ right_arm | held-out subj 20 | F1=0.0109
→ right_arm | held-out subj 21 | F1=0.0112
→ right_arm | held-out subj 3 | F1=0.0172
→ right_arm | held-out subj 4 | F1=0.0178
→ right_arm | held-out subj 5 | F1=0.0189
→ right_arm | held-out subj 6 | F1=0.0117
→ right_arm | held-out subj 7 | F1=0.0156
→ right_arm | held-out subj 8 | F1=0.0301
→ right_arm | held-out subj 9 | F1=0.0139
=== right_arm LOSO mean F1: 0.0160 ===
LOSO scores per location:

t### 6. Train on full data & predict test set (creates `submission.csv`)

In [9]:

def predict_test(model, df, scaler):
    ds = WearWindowDataset(df, scaler, train=False, augment=False)
    dl = DataLoader(ds, batch_size=BATCH_SZ, shuffle=False, num_workers=NUM_WORKERS)
    model.eval(); preds = []
    with torch.no_grad():
        for x in dl:
            x = x.to(DEVICE)
            logits = model(x)
            preds.append(logits.argmax(1).cpu().numpy())
    return np.concatenate(preds)

subs = []
for loc in ['right_arm']:# LOCATION_IDS:
    train_df = pd.read_csv(f'train_{loc}.csv')
    test_df  = pd.read_csv(f'test_{loc}.csv')
    scaler   = StandardScaler().fit(train_df.filter(regex='_acc_'))
    train_ds = WearWindowDataset(train_df, scaler, train=True, augment=False)
    train_loader = DataLoader(train_ds, batch_size=BATCH_SZ, shuffle=True)
    n_classes = train_df['label'].max() + 1
    class_weights = compute_class_weight('balanced', classes=np.arange(n_classes), y=train_ds.y)
    model = ShallowDeepConvLSTM(n_classes).to(DEVICE)
    optim = AdamW(model.parameters(), lr=LR)
    scheduler = CosineAnnealingLR(optim, T_max=EPOCHS)
    crit = FocalLoss(alpha=torch.tensor(class_weights, device=DEVICE))
    for epoch in range(1, EPOCHS+1):
        run_epoch(model, train_loader, optim, scheduler, crit)
    preds = predict_test(model, test_df, scaler)
    sub_df = pd.DataFrame({'row_id': test_df['row_id'], 'label': preds})
    subs.append(sub_df)

submission = pd.concat(subs).sort_values('row_id')
submission.to_csv('submission.csv', index=False)
print('Saved submission.csv')


FileNotFoundError: [Errno 2] No such file or directory: 'train_right_arm.csv'

In [12]:
import torch
from pathlib import Path
from typing import Union, Sequence

def merge_fold_checkpoints(
    fold_dir: Union[str, Path],
    pattern: str = "model_right_arm_fold*.pt",
    output_path: Union[str, Path] = "model_right_arm_averaged.pt"
) -> None:
    """
    Averages the weights of all fold checkpoints matching <pattern> in <fold_dir>
    and writes a single averaged checkpoint to <output_path>.

    fold_dir:     directory containing per-fold .pt files
    pattern:      glob pattern to match your fold files
    output_path:  path for the merged checkpoint
    """
    fold_dir   = Path(fold_dir)
    output_path = Path(output_path)

    torch.serialization.add_safe_globals(["numpy.core.multiarray._reconstruct"])

    fold_files = sorted(fold_dir.glob(pattern))
    if not fold_files:
        raise FileNotFoundError(f"No files matching {pattern} in {fold_dir}")

    state_dicts = []
    for p in fold_files:
        ckpt = torch.load(p, map_location="cpu", weights_only=False)
        sd   = ckpt.get("state_dict", ckpt)
        state_dicts.append(sd)

    avg_sd = {}
    keys = state_dicts[0].keys()
    for key in keys:
        # stack along new dim=0 then mean
        stacked = torch.stack([sd[key].float() for sd in state_dicts], dim=0)
        avg_sd[key] = stacked.mean(dim=0)

    torch.save({"state_dict": avg_sd}, output_path)
    print(f"✅ Merged {len(fold_files)} checkpoints → {output_path}")


In [13]:
merge_fold_checkpoints(
    fold_dir="work1",
    pattern="model_right_arm_fold*.pt",
    output_path="work1/model_right_arm_averaged.pt"
)


✅ Merged 22 checkpoints → work1/model_right_arm_averaged.pt


In [14]:
ckpt = torch.load("work1/model_right_arm_averaged.pt", map_location=DEVICE)
model = ShallowDeepConvLSTM(n_classes=len(label_map)).to(DEVICE)
model.load_state_dict(ckpt["state_dict"])
model.eval()


ShallowDeepConvLSTM(
  (conv): Sequential(
    (0): Conv1d(3, 64, kernel_size=(5,), stride=(1,), padding=(2,))
    (1): ReLU()
    (2): Conv1d(64, 64, kernel_size=(5,), stride=(1,), padding=(2,))
    (3): ReLU()
    (4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (lstm): LSTM(64, 128, batch_first=True)
  (embed): Sequential(
    (0): Dropout(p=0.3, inplace=False)
    (1): Linear(in_features=128, out_features=64, bias=True)
    (2): ReLU(inplace=True)
  )
  (head): Linear(in_features=64, out_features=19, bias=True)
)

In [20]:
class WearWindowDataset(Dataset):
    def __init__(self, df, scaler=None, train=True, augment=True):
        self.train   = train
        self.augment = augment and train

        acc_x_cols = [c for c in df.columns if c.endswith('_acc_x')]
        if acc_x_cols:
            loc = acc_x_cols[0].split('_acc_')[0]
            axes = [f"{loc}_acc_{ax}" for ax in ('x','y','z')]
        else:
            axes = [c for c in df.columns if c.endswith('_axis')]
            if sorted(axes) != ['x_axis', 'y_axis', 'z_axis']:
                raise ValueError(f"Unrecognized axis columns: {axes}")

        feats = df[axes].values.astype('float32')   # (N_rows, 3)
        labels = df['label'].values.astype('int64') if 'label' in df.columns else None

        X, y = [], []
        for i in range(0, len(feats)-WIN_SIZE+1, STRIDE):
            w = feats[i:i+WIN_SIZE]
            if np.isnan(w).any():
                continue
            X.append(w)
            if labels is not None:
                y.append(np.bincount(labels[i:i+WIN_SIZE]).argmax())

        self.X = np.stack(X)                        # (N_windows, 50, 3)
        self.y = np.array(y) if labels is not None else None

        if scaler is not None:
            flat = self.X.reshape(-1, 3)
            flat = scaler.transform(flat)
            self.X = flat.reshape(-1, WIN_SIZE, 3)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        # (50,3) -> (3,50)
        x = torch.tensor(self.X[idx]).permute(1, 0)
        if self.augment:
            x = jitter(x)
            x = scaling(x)
        if self.y is None:
            return x
        return x, torch.tensor(self.y[idx])


In [24]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
import ast
from collections import Counter

ckpt  = torch.load("work1/model_right_arm_averaged.pt", map_location=DEVICE)
model = ShallowDeepConvLSTM(n_classes=len(label_map)).to(DEVICE)
model.load_state_dict(ckpt["state_dict"])
model.eval()

df_train = pd.read_csv(WORK_DIR/"train_right_arm.csv")
train_axes = [c for c in df_train.columns if c.endswith("_acc_x")]
loc        = train_axes[0].split("_acc_")[0]
scaler     = StandardScaler().fit(
    df_train[[f"{loc}_acc_{ax}" for ax in ("x","y","z")]]
)

df_test = pd.read_csv(TEST_CSV)

results = []

WIN = 50
STRIDE = 25

for _, row in df_test.iterrows():
    rid    = row["id"]
    x_full = np.array(ast.literal_eval(row["x_axis"]), dtype=np.float32)
    y_full = np.array(ast.literal_eval(row["y_axis"]), dtype=np.float32)
    z_full = np.array(ast.literal_eval(row["z_axis"]), dtype=np.float32)

    L = len(x_full)
    window_preds = []

    for i in range(0, L - WIN + 1, STRIDE):
        w = np.stack([x_full[i:i+WIN],
                      y_full[i:i+WIN],
                      z_full[i:i+WIN]], axis=1)  # (50,3)
        flat = scaler.transform(w)            # (50,3)
        win  = torch.tensor(flat.T).unsqueeze(0).to(DEVICE)  # (1,3,50)
        with torch.no_grad():
            logits = model(win)               # (1, n_classes)
        window_preds.append(int(logits.argmax(1).cpu().item()))

    if not window_preds:
        w = np.stack([x_full, y_full, z_full], axis=1)
        flat = scaler.transform(
            np.pad(w, ((0, max(0, WIN-L)), (0,0)), mode='edge')
        )
        win  = torch.tensor(flat.T).unsqueeze(0).to(DEVICE)
        with torch.no_grad():
            window_preds.append(int(model(win).argmax(1).cpu().item()))

    label = Counter(window_preds).most_common(1)[0][0]
    results.append((rid, label))

out_df = pd.DataFrame(results, columns=["id","label"])
out_df.to_csv("result.csv", index=False)
print("✅ Wrote result.csv with", len(out_df), "rows.")


✅ Wrote result.csv with 48936 rows.
